In [100]:
import pandas as pd
import torch

from pathlib import Path
import sys
sys.path.append("../../modules/yolov5/")

%load_ext autoreload
%autoreload 2

In [2]:
data_dir = Path("../../data/raw")
images_dir = data_dir / "train_images"

In [5]:
df_train = pd.read_pickle(data_dir / "train_transformed.pickle")
df_train.sample(3)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,path,bboxes,n_bboxes
21071,2,29859,8330,558,2-8330,[],/home/isabella/code/competitions/great_barrier...,[],0
522,0,45518,915,42,0-915,[],/home/isabella/code/competitions/great_barrier...,[],0
7107,1,60510,399,399,1-399,[],/home/isabella/code/competitions/great_barrier...,[],0


## Load sample image

In [43]:
from PIL import Image

image = Image.open(df_train.iloc[10].path)
width, height = image.size

## Load base model

In [32]:
device = 'cpu'

weights_path = "../../weights/yolov5s.pt"
checkpoint = torch.load(weights_path, map_location=device)
model = checkpoint['model']

## Freeze model layers

In [31]:
def freeze_layers(model, n: int):
    freeze_layer_names = [f"model.{i}." for i in range(n)]
    for k, v in model.named_parameters():
        v.requires_grad = True
        if any(x in k for x in freeze_layer_names):
            print("freeze", k)
            v.requires_grad = False
            
freeze_layers(model, 0)

## Verify image is divisible by stride

In [51]:
from utils.general import check_img_size

grid_size = max(int(model.stride.max()), 32)
new_image_size = check_img_size(640, grid_size)

# DataLoaders and datasets

In [62]:
from pprint import pprint
import yaml

split_path = Path("../../data/splits/s1")

with open(split_path / "data.yaml") as f:
    data = yaml.load(f, yaml.Loader)
    
pprint(data)

{'names': ['starfish'],
 'nc': 1,
 'train': '/home/isabella/code/competitions/great_barrier_reef/data/splits/s1/train.txt',
 'val': '/home/isabella/code/competitions/great_barrier_reef/data/splits/s1/valid.txt'}


In [73]:
from utils.datasets import create_dataloader

train_dataloader, train_dataset = create_dataloader(
    path=data['train'],
    imgsz=new_image_size,
    batch_size=4, 
    stride=grid_size, 
    single_cls=True, 
    augment=True, 
    rect=False, 
    workers=2,
    prefix="train: ", 
    shuffle=True
)

val_dataloader, val_dataset = create_dataloader(
    path=data['val'],
    imgsz=new_image_size,
    batch_size=4, 
    stride=grid_size, 
    single_cls=True, 
    augment=False, 
    workers=2,
    prefix="valid: ", 
    shuffle=True
)

train: Scanning '/home/isabella/code/competitions/great_barrier_reef/data/splits/s1/train.cache' images and labels... 46 found, 4 missing, 0 empty, 0 corrupted: 100%|███████████████████████| 50/50 [00:00<?, ?it/s]
valid: Scanning '/home/isabella/code/competitions/great_barrier_reef/data/splits/s1/valid.cache' images and labels... 9 found, 29 missing, 0 empty, 0 corrupted: 100%|███████████████████████| 38/38 [00:00<?, ?it/s]


In [89]:
nl = model.model[-1].nl  # number of detection layers
model.nc = 1  # number of classes
model.hyp['obj'] *= (new_image_size / 640) ** 2 * 3 / nl  # scale to image size and layers
model.names = data['names']

In [98]:
from val import process_batch

detections = torch.Tensor([
    [0, 0, 0.5, 0.5, 0.7, 0],
    [0, 0, 0.4, 0.5, 0.7, 0],
    [0, 0, 0.3, 0.5, 0.7, 0],
])

labels = torch.Tensor([
    [0, 0, 0, 0.5, 0.4],
    [1, 0.5, 0.5, 0.5, 0.4],
])

iouv = torch.linspace(0.3, 0.8, 11)

process_batch(detections, labels, iouv).numpy().astype(int)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [101]:
from utils.metrics import box_iou

iou = box_iou(labels[:, 1:], detections[:, :4])

In [107]:
x = torch.where((iou >= iouv[0]) & (labels[:, 0:1] == detections[:, 5]))

In [108]:
matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()

In [109]:
matches

array([[          0,           0,         0.8],
       [          0,           1,     0.66667],
       [          0,           2,     0.52174]], dtype=float32)

In [104]:
iou

tensor([[0.80000, 0.66667, 0.52174],
        [0.00000, 0.00000, 0.00000]])